In [1]:
# Andres Sebastian Salazar Alturo
# Candidate number: 276209

## Env alias - mlcw

## Utilities
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Food trade indicators pre-process

In [2]:
## Import food trade indicators data

food_trade_indicators_file_path = "data/Food trade indicators - FAOSTAT_data_en_2-22-2024.csv"

## Read the csv file and transform to dataframe
food_trade_indicators_df = pd.read_csv(food_trade_indicators_file_path)

## Display the 5 rows of the DataFrame
food_trade_indicators_df.head(2)

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code (CPC),Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,TCL,Crops and livestock products,4,Afghanistan,5622,Import Value,F1888,Cereals and Preparations,1991,1991,1000 USD,41600.0,A,Official figure,NaN
1,TCL,Crops and livestock products,4,Afghanistan,5622,Import Value,F1888,Cereals and Preparations,1992,1992,1000 USD,25600.0,E,Estimated value,NaN


In [3]:
# Assuming 'column_name' is the name of the column you're interested in
unique_values = food_trade_indicators_df['Element'].unique()

# Print the unique values
print(unique_values)

['Import Value' 'Export Value']


### Export target value

In [4]:
## Get the export value as the target

# Filter the DataFrame - .copy() method ensures that the modifications to export_value_target do not affect the original dataframe
# export_value_target contains only the rows where the Element column is 'Export Value'
export_value_target = food_trade_indicators_df[food_trade_indicators_df['Element'] == 'Export Value'].copy()
export_value_target.head(2)

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code (CPC),Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
19,TCL,Crops and livestock products,4,Afghanistan,5922,Export Value,F1888,Cereals and Preparations,2009,2009,1000 USD,15.0,A,Official figure,NaN
21,TCL,Crops and livestock products,4,Afghanistan,5922,Export Value,F1888,Cereals and Preparations,2010,2010,1000 USD,54.0,A,Official figure,NaN


In [5]:
columns_dropped_food_trade = ["Domain Code", "Domain", "Area Code (M49)", "Element Code", "Element", "Item Code (CPC)",
                              "Item", "Year Code", "Flag", "Flag Description", "Note"]

export_value_filtered_df = export_value_target.drop(columns=columns_dropped_food_trade)
export_value_filtered_df.head(2)

,Area,Year,Unit,Value
19,Afghanistan,2009,1000 USD,15.0
21,Afghanistan,2010,1000 USD,54.0


In [6]:
# Assuming 'column_name' is the name of the column you're interested in
unique_values = export_value_filtered_df['Year'].unique()

# Print the unique values
print(unique_values)

[2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022
 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004
 2005 2006 2007 2008]


### Handle the zero values with the average value of the area to keep as much information as possible

In [7]:
# Calculate the average 'Value' for each 'Area', excluding zeros
average_values = export_value_filtered_df[export_value_filtered_df['Value'] != 0].groupby('Area')['Value'].mean().reset_index()

# Map average 'Value' to each 'Area' where 'Value' is zero
export_value_filtered_df['Value'] = export_value_filtered_df.apply(
    lambda row: average_values[average_values['Area'] == row['Area']]['Value'].iloc[0] if row['Value'] == 0 else row['Value'],
    axis=1
)

# Remove duplicate years for each country to ensure unique year entries per country
export_data = export_value_filtered_df.drop_duplicates(subset=['Area', 'Year'])

# Sort the DataFrame by 'Area' and 'Year' in ascending order
export_data_target_df = export_data.sort_values(by=['Area', 'Year'])
export_data_target_df

,Area,Year,Unit,Value
128,Afghanistan,1991,1000 USD,50554.413314
130,Afghanistan,1992,1000 USD,50554.413314
132,Afghanistan,1993,1000 USD,50554.413314
134,Afghanistan,1994,1000 USD,50554.413314
136,Afghanistan,1995,1000 USD,50554.413314
...,...,...,...,...
141027,Zimbabwe,2018,1000 USD,7562.860000
141029,Zimbabwe,2019,1000 USD,9134.560000
141031,Zimbabwe,2020,1000 USD,17120.490000
141033,Zimbabwe,2021,1000 USD,16227.950000


In [8]:
## Export the export value data to csv
export_data_target_df.to_csv('my_data/export_data_target.csv', index = False)

### Process "Import price" feature

In [9]:
## Get the import value as feature

# Filter the DataFrame - .copy() method ensures that the modifications to export_value_target do not affect the original dataframe
# export_value_target contains only the rows where the Element column is 'Export Value'
import_value = food_trade_indicators_df[food_trade_indicators_df['Element'] == 'Import Value'].copy()
import_value.head(2)

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code (CPC),Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,TCL,Crops and livestock products,4,Afghanistan,5622,Import Value,F1888,Cereals and Preparations,1991,1991,1000 USD,41600.0,A,Official figure,NaN
1,TCL,Crops and livestock products,4,Afghanistan,5622,Import Value,F1888,Cereals and Preparations,1992,1992,1000 USD,25600.0,E,Estimated value,NaN


In [10]:
columns_dropped_food_trade = ["Domain Code", "Domain", "Area Code (M49)", "Element Code", "Element", "Item Code (CPC)",
                              "Item", "Year Code", "Flag", "Flag Description", "Note"]

import_value_filtered_df = import_value.drop(columns=columns_dropped_food_trade)
import_value_filtered_df.head(2)

,Area,Year,Unit,Value
0,Afghanistan,1991,1000 USD,41600.0
1,Afghanistan,1992,1000 USD,25600.0


### Handle the zero values with the average value of the area to keep as much information as possible

In [11]:
# Calculate the average 'Value' for each 'Area', excluding zeros
average_values = import_value_filtered_df[import_value_filtered_df['Value'] != 0].groupby('Area')['Value'].mean().reset_index()

# Map average 'Value' to each 'Area' where 'Value' is zero
import_value_filtered_df['Value'] = import_value_filtered_df.apply(
    lambda row: average_values[average_values['Area'] == row['Area']]['Value'].iloc[0] if row['Value'] == 0 else row['Value'],
    axis=1
)

# Remove duplicate years for each country to ensure unique year entries per country
import_data = import_value_filtered_df.drop_duplicates(subset=['Area', 'Year'])

# Sort the DataFrame by 'Area' and 'Year' in ascending order
import_data_target_df = import_data.sort_values(by=['Area', 'Year'])
import_data_target_df

,Area,Year,Unit,Value
0,Afghanistan,1991,1000 USD,41600.00
1,Afghanistan,1992,1000 USD,25600.00
2,Afghanistan,1993,1000 USD,40000.00
3,Afghanistan,1994,1000 USD,25700.00
4,Afghanistan,1995,1000 USD,37720.00
...,...,...,...,...
141026,Zimbabwe,2018,1000 USD,293431.28
141028,Zimbabwe,2019,1000 USD,153705.24
141030,Zimbabwe,2020,1000 USD,601835.56
141032,Zimbabwe,2021,1000 USD,407807.82


In [12]:
## Export the import data value to csv
import_data_target_df.to_csv('my_data/import_data_value.csv', index = False)

## Consumer price indicators pre-processing
### Food inflation feature

In [13]:
## Import Consumer prices indicators data

consumer_prices_indicators_file_path = "data/Consumer prices indicators - FAOSTAT_data_en_2-22-2024.csv"

## Read the csv file and transform to dataframe
consumer_prices_indicators_df = pd.read_csv(consumer_prices_indicators_file_path)

## Display the 5 rows of the DataFrame
consumer_prices_indicators_df.head(2)

,Domain Code,Domain,Area Code (M49),Area,Year Code,Year,Item Code,Item,Months Code,Months,Element Code,Element,Unit,Value,Flag,Flag Description,Note
0,CP,Consumer Price Indices,4,Afghanistan,2000,2000,23013,"Consumer Prices, Food Indices (2015 = 100)",7001,January,6125,Value,NaN,24.356332,I,Imputed value,base year is 2015
1,CP,Consumer Price Indices,4,Afghanistan,2000,2000,23013,"Consumer Prices, Food Indices (2015 = 100)",7002,February,6125,Value,NaN,23.636242,I,Imputed value,base year is 2015


In [14]:
# Assuming 'column_name' is the name of the column you're interested in
unique_values = consumer_prices_indicators_df['Item'].unique()

# Print the unique values
print(unique_values)

['Consumer Prices, Food Indices (2015 = 100)' 'Food price inflation']


In [15]:
## Get the food price inflation as feature

# Filter the DataFrame - .copy() method ensures that the modifications to export_value_target do not affect the original dataframe
# export_value_target contains only the rows where the Element column is 'Export Value'
food_inflation_value = consumer_prices_indicators_df[consumer_prices_indicators_df['Item'] == 'Food price inflation'].copy()
food_inflation_value.head(2)

,Domain Code,Domain,Area Code (M49),Area,Year Code,Year,Item Code,Item,Months Code,Months,Element Code,Element,Unit,Value,Flag,Flag Description,Note
285,CP,Consumer Price Indices,4,Afghanistan,2001,2001,23014,Food price inflation,7001,January,6121,Value,%,22.943765,E,Estimated value,NaN
286,CP,Consumer Price Indices,4,Afghanistan,2001,2001,23014,Food price inflation,7002,February,6121,Value,%,24.357903,E,Estimated value,NaN


In [16]:
columns_dropped_consumer_price_indicators = ["Domain Code", "Domain", "Area Code (M49)", "Element Code", "Element", "Unit","Item Code", "Months Code", "Months",
                              "Item", "Year Code", "Flag", "Flag Description", "Note"]

food_inflation_filtered_df = food_inflation_value.drop(columns=columns_dropped_consumer_price_indicators)
food_inflation_filtered_df.head(2)

,Area,Year,Value
285,Afghanistan,2001,22.943765
286,Afghanistan,2001,24.357903


### Handle the zero values with the average value of the area to keep as much information as possible

In [17]:
# Calculate the average 'Value' for each 'Area', excluding zeros
average_values = food_inflation_filtered_df[food_inflation_filtered_df['Value'] != 0].groupby('Area')['Value'].mean().reset_index()

# Map average 'Value' to each 'Area' where 'Value' is zero
food_inflation_filtered_df['Value'] = food_inflation_filtered_df.apply(
    lambda row: average_values[average_values['Area'] == row['Area']]['Value'].iloc[0] if row['Value'] == 0 else row['Value'],
    axis=1
)

# Remove duplicate years for each country to ensure unique year entries per country
food_inflation_data = food_inflation_filtered_df.drop_duplicates(subset=['Area', 'Year'])

# Sort the DataFrame by 'Area' and 'Year' in ascending order
food_inflation_df = food_inflation_data.sort_values(by=['Area', 'Year'])
food_inflation_df

,Area,Year,Value
285,Afghanistan,2001,22.943765
297,Afghanistan,2002,11.612646
309,Afghanistan,2003,19.585062
321,Afghanistan,2004,8.592641
333,Afghanistan,2005,12.390806
...,...,...,...
1059,Åland Islands,2019,1.942669
1071,Åland Islands,2020,3.064903
1083,Åland Islands,2021,-1.115151
1095,Åland Islands,2022,3.537360


In [18]:
## Export the import data value to csv
food_inflation_df.to_csv('my_data/food_inflation_data.csv', index = False)

## Food balances pre-processing
### Export quantity feature

In [19]:
## Import food balances indicators data

food_balances_indicators_file_path = "data/Food balances indicators - FAOSTAT_data_en_2-22-2024.csv"

## Read the csv file and transform to dataframe
food_balances_indicators_df = pd.read_csv(food_balances_indicators_file_path)

## Display the 5 rows of the DataFrame
food_balances_indicators_df.head(2)

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code (FBS),Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,FBS,Food Balances (2010-),4,Afghanistan,5611,Import Quantity,S2905,Cereals - Excluding Beer,2010,2010,1000 t,2000.0,E,Estimated value
1,FBS,Food Balances (2010-),4,Afghanistan,5611,Import Quantity,S2905,Cereals - Excluding Beer,2011,2011,1000 t,2448.0,E,Estimated value


In [20]:
# Assuming 'column_name' is the name of the column you're interested in
unique_values = food_balances_indicators_df['Element'].unique()

# Print the unique values
print(unique_values)

['Import Quantity' 'Export Quantity' 'Losses' 'Other uses (non-food)'
 'Food']


In [21]:
## Get the export value as features
export_quantity_df = food_balances_indicators_df[food_balances_indicators_df['Element'] == 'Export Quantity'].copy()
export_quantity_df.head(2)

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code (FBS),Item,Year Code,Year,Unit,Value,Flag,Flag Description
12,FBS,Food Balances (2010-),4,Afghanistan,5911,Export Quantity,S2905,Cereals - Excluding Beer,2010,2010,1000 t,0.0,E,Estimated value
13,FBS,Food Balances (2010-),4,Afghanistan,5911,Export Quantity,S2905,Cereals - Excluding Beer,2011,2011,1000 t,0.0,E,Estimated value


In [22]:
columns_dropped_food_balances_indicators = ["Domain Code", "Domain", "Area Code (M49)", "Element Code", "Element", "Item Code (FBS)",
                              "Item", "Year Code", "Flag", "Flag Description"]

export_quantity_df_filtered = export_quantity_df.drop(columns=columns_dropped_food_balances_indicators)
export_quantity_df_filtered

,Area,Year,Unit,Value
12,Afghanistan,2010,1000 t,0.00
13,Afghanistan,2011,1000 t,0.00
14,Afghanistan,2012,1000 t,0.00
15,Afghanistan,2013,1000 t,0.00
16,Afghanistan,2014,1000 t,2.00
...,...,...,...,...
148012,Zimbabwe,2017,1000 t,3.85
148013,Zimbabwe,2018,1000 t,4.94
148014,Zimbabwe,2019,1000 t,5.53
148015,Zimbabwe,2020,1000 t,5.53


### Handle the zero values with the average value of the area to keep as much information as possible

In [23]:
# Assuming export_quantity_df_filtered is your DataFrame loaded from a CSV or another source
# Calculate the average 'Value' for each 'Area', excluding zeros
average_values = export_quantity_df_filtered[export_quantity_df_filtered['Value'] != 0]
average_values = average_values.groupby('Area')['Value'].mean()

# Create a dictionary from the series for quicker access
average_values_dict = average_values.to_dict()

# Replace zero values with the average of their respective 'Area'
export_quantity_df_filtered['Value'] = export_quantity_df_filtered.apply(
    lambda row: average_values_dict.get(row['Area'], 0) if row['Value'] == 0 else row['Value'], axis=1
)

# Remove duplicate years for each country to ensure unique year entries per country
export_quantity_data = export_quantity_df_filtered.drop_duplicates(subset=['Area', 'Year'])

# Sort the DataFrame by 'Area' and 'Year' in ascending order
export_quantity_df = export_quantity_data.sort_values(by=['Area', 'Year'])
export_quantity_df

,Area,Year,Unit,Value
12,Afghanistan,2010,1000 t,79.663265
13,Afghanistan,2011,1000 t,79.663265
14,Afghanistan,2012,1000 t,79.663265
15,Afghanistan,2013,1000 t,79.663265
16,Afghanistan,2014,1000 t,2.000000
...,...,...,...,...
147260,Zimbabwe,2017,1000 t,25.000000
147261,Zimbabwe,2018,1000 t,14.000000
147262,Zimbabwe,2019,1000 t,16.000000
147263,Zimbabwe,2020,1000 t,20.000000


In [24]:
## Export the import data value to csv
export_quantity_df.to_csv('my_data/export_quantity_data.csv', index = False)

### Import quantity feature

In [25]:
## Get the import value as feature
import_quantity_df = food_balances_indicators_df[food_balances_indicators_df['Element'] == 'Import Quantity'].copy()
import_quantity_df.head(2)

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code (FBS),Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,FBS,Food Balances (2010-),4,Afghanistan,5611,Import Quantity,S2905,Cereals - Excluding Beer,2010,2010,1000 t,2000.0,E,Estimated value
1,FBS,Food Balances (2010-),4,Afghanistan,5611,Import Quantity,S2905,Cereals - Excluding Beer,2011,2011,1000 t,2448.0,E,Estimated value


In [26]:
# Assuming 'column_name' is the name of the column you're interested in
unique_values = import_quantity_df['Unit'].unique()

# Print the unique values
print(unique_values)

['1000 t']


In [27]:
columns_dropped_food_balances_indicators = ["Domain Code", "Domain", "Area Code (M49)", "Element Code", "Element", "Item Code (FBS)",
                              "Item", "Year Code", "Flag", "Flag Description"]

import_quantity_df_filtered = import_quantity_df.drop(columns=columns_dropped_food_balances_indicators)
import_quantity_df_filtered.head(2)

,Area,Year,Unit,Value
0,Afghanistan,2010,1000 t,2000.0
1,Afghanistan,2011,1000 t,2448.0


In [28]:
# Calculate the average 'Value' for each 'Area', excluding zeros
average_values = import_quantity_df_filtered[import_quantity_df_filtered['Value'] != 0].groupby('Area')['Value'].mean().reset_index()

# Map average 'Value' to each 'Area' where 'Value' is zero
import_quantity_df_filtered['Value'] = import_quantity_df_filtered.apply(
    lambda row: average_values[average_values['Area'] == row['Area']]['Value'].iloc[0] if row['Value'] == 0 else row['Value'],
    axis=1
)

# Remove duplicate years for each country to ensure unique year entries per country
import_quantity_data = import_quantity_df_filtered.drop_duplicates(subset=['Area', 'Year'])

# Sort the DataFrame by 'Area' and 'Year' in ascending order
import_quantity_df = import_quantity_data.sort_values(by=['Area', 'Year'])
import_quantity_df

,Area,Year,Unit,Value
0,Afghanistan,2010,1000 t,2000.0
1,Afghanistan,2011,1000 t,2448.0
2,Afghanistan,2012,1000 t,2001.0
3,Afghanistan,2013,1000 t,2155.0
4,Afghanistan,2014,1000 t,1840.0
...,...,...,...,...
147248,Zimbabwe,2017,1000 t,901.0
147249,Zimbabwe,2018,1000 t,743.0
147250,Zimbabwe,2019,1000 t,671.0
147251,Zimbabwe,2020,1000 t,1724.0


In [29]:
## Export the import quantity data value to csv
import_quantity_df.to_csv('my_data/import_quantity_data.csv', index = False)

### Crop losses

In [30]:
## Get crop losses as feature

crop_losses_df = food_balances_indicators_df[food_balances_indicators_df['Element'] == 'Losses'].copy()
crop_losses_df.head(2)

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code (FBS),Item,Year Code,Year,Unit,Value,Flag,Flag Description
24,FBS,Food Balances (2010-),4,Afghanistan,5123,Losses,S2905,Cereals - Excluding Beer,2010,2010,1000 t,837.0,E,Estimated value
25,FBS,Food Balances (2010-),4,Afghanistan,5123,Losses,S2905,Cereals - Excluding Beer,2011,2011,1000 t,647.0,E,Estimated value


In [31]:
# Assuming 'column_name' is the name of the column you're interested in
unique_values = crop_losses_df['Unit'].unique()

# Print the unique values
print(unique_values)

['1000 t']


In [32]:
columns_dropped_food_balances_indicators = ["Domain Code", "Domain", "Area Code (M49)", "Element Code", "Element", "Item Code (FBS)",
                              "Item", "Year Code", "Flag", "Flag Description"]

crop_losses_filtered_df = crop_losses_df.drop(columns=columns_dropped_food_balances_indicators)
crop_losses_filtered_df

,Area,Year,Unit,Value
24,Afghanistan,2010,1000 t,837.0
25,Afghanistan,2011,1000 t,647.0
26,Afghanistan,2012,1000 t,911.0
27,Afghanistan,2013,1000 t,936.0
28,Afghanistan,2014,1000 t,963.0
...,...,...,...,...
147932,Zimbabwe,2017,1000 t,2.0
147933,Zimbabwe,2018,1000 t,2.0
147934,Zimbabwe,2019,1000 t,2.0
147935,Zimbabwe,2020,1000 t,2.0


In [33]:
# Assuming crop_losses_filtered_df is your DataFrame loaded from a CSV or another source
# Calculate the average 'Value' for each 'Area', excluding zeros
average_values = crop_losses_filtered_df[crop_losses_filtered_df['Value'] != 0]
average_values = average_values.groupby('Area')['Value'].mean()

# Create a dictionary from the series for quicker access
average_values_dict = average_values.to_dict()

# Replace zero values with the average of their respective 'Area'
crop_losses_filtered_df['Value'] = crop_losses_filtered_df.apply(
    lambda row: average_values_dict.get(row['Area'], 0) if row['Value'] == 0 else row['Value'], axis=1
)

# Remove duplicate years for each country to ensure unique year entries per country
crop_losses_data = crop_losses_filtered_df.drop_duplicates(subset=['Area', 'Year'])

# Sort the DataFrame by 'Area' and 'Year' in ascending order
crop_losses_df = crop_losses_data.sort_values(by=['Area', 'Year'])
crop_losses_df

,Area,Year,Unit,Value
24,Afghanistan,2010,1000 t,837.0
25,Afghanistan,2011,1000 t,647.0
26,Afghanistan,2012,1000 t,911.0
27,Afghanistan,2013,1000 t,936.0
28,Afghanistan,2014,1000 t,963.0
...,...,...,...,...
147272,Zimbabwe,2017,1000 t,80.0
147273,Zimbabwe,2018,1000 t,81.0
147274,Zimbabwe,2019,1000 t,33.0
147275,Zimbabwe,2020,1000 t,113.0


In [34]:
## Export the crop losses data value to csv
crop_losses_df.to_csv('my_data/crop_losses_data.csv', index = False)

## Exchange pre-processing

In [35]:
## Import exchange currency per dollar data

exchange_file_path = "data/Exchange rate - FAOSTAT_data_en_2-22-2024.csv"

## Read the csv file and transform to dataframe
exchange_original_df = pd.read_csv(exchange_file_path)

## Display the 5 rows of the DataFrame
exchange_original_df.head(2)

,Domain Code,Domain,Area Code (M49),Area,ISO Currency Code (FAO),Currency,Element Code,Element,Year Code,Year,Months Code,Months,Unit,Value,Flag,Flag Description
0,PE,Exchange rates,4,Afghanistan,AFA,Afghani,LCU,Local currency units per USD,1980,1980,7001,January,NaN,44.129167,X,Figure from international organizations
1,PE,Exchange rates,4,Afghanistan,AFA,Afghani,LCU,Local currency units per USD,1980,1980,7002,February,NaN,44.129167,X,Figure from international organizations


In [36]:
# Assuming 'column_name' is the name of the column you're interested in
unique_values = exchange_original_df['Element'].unique()

# Print the unique values
print(unique_values)

['Local currency units per USD']


In [37]:
## Get the Indicator as feature
exchange_original_df = exchange_original_df[exchange_original_df['Element'] == 'Local currency units per USD'].copy()
exchange_original_df.head(2)

,Domain Code,Domain,Area Code (M49),Area,ISO Currency Code (FAO),Currency,Element Code,Element,Year Code,Year,Months Code,Months,Unit,Value,Flag,Flag Description
0,PE,Exchange rates,4,Afghanistan,AFA,Afghani,LCU,Local currency units per USD,1980,1980,7001,January,NaN,44.129167,X,Figure from international organizations
1,PE,Exchange rates,4,Afghanistan,AFA,Afghani,LCU,Local currency units per USD,1980,1980,7002,February,NaN,44.129167,X,Figure from international organizations


In [38]:
columns_dropped_exchange = ["Domain Code", "Domain", "Area Code (M49)", "Element Code", "Element", "ISO Currency Code (FAO)",
                              "Year Code", "Months Code", "Months", "Flag", "Flag Description", "Unit"]

exchange_final_df = exchange_original_df.drop(columns=columns_dropped_exchange)
exchange_final_df.head(2)

,Area,Currency,Year,Value
0,Afghanistan,Afghani,1980,44.129167
1,Afghanistan,Afghani,1980,44.129167


In [39]:
exchange_column_order = ['Area', 'Year', 'Currency', 'Value']
exchange_final_df = exchange_final_df[exchange_column_order]
exchange_final_df

,Area,Year,Currency,Value
0,Afghanistan,1980,Afghani,44.129167
1,Afghanistan,1980,Afghani,44.129167
2,Afghanistan,1980,Afghani,44.129167
3,Afghanistan,1980,Afghani,44.129167
4,Afghanistan,1980,Afghani,44.129167
...,...,...,...,...
103271,Zimbabwe,2022,Zimbabwe Dollar (old),597.970000
103272,Zimbabwe,2022,Zimbabwe Dollar (old),628.716452
103273,Zimbabwe,2023,Zimbabwe Dollar (old),981.370229
103274,Zimbabwe,2023,Zimbabwe Dollar (old),1439.613438


In [40]:
# Assuming exchange_final_df is your DataFrame loaded from a CSV or another source
# Calculate the average 'Value' for each 'Area', excluding zeros
average_values = exchange_final_df[exchange_final_df['Value'] != 0]
average_values = average_values.groupby('Area')['Value'].mean()

# Create a dictionary from the series for quicker access
average_values_dict = average_values.to_dict()

# Replace zero values with the average of their respective 'Area'
exchange_final_df['Value'] = exchange_final_df.apply(
    lambda row: average_values_dict.get(row['Area'], 0) if row['Value'] == 0 else row['Value'], axis=1
)

# Remove duplicate years for each country to ensure unique year entries per country
exhange_data = exchange_final_df.drop_duplicates(subset=['Area', 'Year'])

# Sort the DataFrame by 'Area' and 'Year' in ascending order
exchange_df = exhange_data.sort_values(by=['Area', 'Year'])
exchange_df

,Area,Year,Currency,Value
0,Afghanistan,1980,Afghani,44.129167
12,Afghanistan,1981,Afghani,49.479902
24,Afghanistan,1982,Afghani,50.599608
36,Afghanistan,1983,Afghani,50.599608
48,Afghanistan,1984,Afghani,50.599606
...,...,...,...,...
737,Åland Islands,2019,Euro,0.875932
749,Åland Islands,2020,Euro,0.900871
761,Åland Islands,2021,Euro,0.821635
773,Åland Islands,2022,Euro,0.883824


In [41]:
## Export the exchange data value to csv
exchange_df.to_csv('my_data/exchange_data.csv', index = False)

## Land use pre-processing
### Agricultural land feature

In [42]:
## Import exchange currency per dollar data

land_use_path = "data/Land use - FAOSTAT_data_en_2-22-2024.csv"

## Read the csv file and transform to dataframe
land_use_df = pd.read_csv(land_use_path)

## Display the 5 rows of the DataFrame
land_use_df.head(2)

C:\Users\andre\AppData\Local\Temp\ipykernel_22404\2330349201.py:6: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  land_use_df = pd.read_csv(land_use_path)


,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,RL,Land Use,4,Afghanistan,5110,Area,6600,Country area,1980,1980,1000 ha,65286.0,A,Official figure,NaN
1,RL,Land Use,4,Afghanistan,5110,Area,6600,Country area,1981,1981,1000 ha,65286.0,A,Official figure,NaN


In [43]:
# Assuming 'column_name' is the name of the column you're interested in
unique_values = land_use_df['Item'].unique()

# Print the unique values
print(unique_values)

['Country area' 'Land area' 'Agriculture' 'Agricultural land' 'Cropland'
 'Arable land' 'Temporary crops' 'Temporary meadows and pastures'
 'Temporary fallow' 'Permanent crops' 'Permanent meadows and pastures'
 'Perm. meadows & pastures - Nat. growing'
 'Land area equipped for irrigation' 'Land area actually irrigated'
 'Agriculture area actually irrigated' 'Farm buildings and Farmyards'
 'Cropland area actually irrigated'
 'Perm. meadows & pastures - Cultivated'
 'Perm. meadows & pastures area actually irrig.'
 'Forestry area actually irrigated']


In [44]:
## Get the Indicator as feature
land_use_agriculturalLand_df = land_use_df[land_use_df['Item'] == 'Agricultural land'].copy()
land_use_agriculturalLand_df.head(2)

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
126,RL,Land Use,4,Afghanistan,5110,Area,6610,Agricultural land,1980,1980,1000 ha,38049.0,A,Official figure,NaN
127,RL,Land Use,4,Afghanistan,5110,Area,6610,Agricultural land,1981,1981,1000 ha,38053.0,A,Official figure,NaN


In [45]:
columns_dropped_land_use = ["Domain Code", "Domain", "Area Code (M49)", "Element Code", "Element", "Item Code", "Item",
                              "Year Code", "Flag", "Flag Description", "Note"]

land_use_agriculturalLand_df = land_use_agriculturalLand_df.drop(columns=columns_dropped_land_use)
land_use_agriculturalLand_df

,Area,Year,Unit,Value
126,Afghanistan,1980,1000 ha,38049.0
127,Afghanistan,1981,1000 ha,38053.0
128,Afghanistan,1982,1000 ha,38054.0
129,Afghanistan,1983,1000 ha,38054.0
130,Afghanistan,1984,1000 ha,38054.0
...,...,...,...,...
97717,Zimbabwe,2017,1000 ha,16200.0
97718,Zimbabwe,2018,1000 ha,16200.0
97719,Zimbabwe,2019,1000 ha,16200.0
97720,Zimbabwe,2020,1000 ha,16200.0


In [46]:
## Export the agricultural land data value to csv
land_use_agriculturalLand_df.to_csv('my_data/agricultural_land.csv', index = False)

### Permanent crops features

In [47]:
## Get the Indicator as feature
land_use_permanentCrops_df = land_use_df[land_use_df['Item'] == 'Permanent crops'].copy()
land_use_permanentCrops_df.head(2)

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
315,RL,Land Use,4,Afghanistan,5110,Area,6650,Permanent crops,1980,1980,1000 ha,139.0,A,Official figure,NaN
316,RL,Land Use,4,Afghanistan,5110,Area,6650,Permanent crops,1981,1981,1000 ha,143.0,A,Official figure,NaN


In [48]:
columns_dropped_land_use = ["Domain Code", "Domain", "Area Code (M49)", "Element Code", "Element", "Item Code", "Item",
                              "Year Code", "Flag", "Flag Description", "Note"]

land_use_permanentCrops_df = land_use_permanentCrops_df.drop(columns=columns_dropped_land_use)
land_use_permanentCrops_df

,Area,Year,Unit,Value
315,Afghanistan,1980,1000 ha,139.0
316,Afghanistan,1981,1000 ha,143.0
317,Afghanistan,1982,1000 ha,144.0
318,Afghanistan,1983,1000 ha,144.0
319,Afghanistan,1984,1000 ha,144.0
...,...,...,...,...
97906,Zimbabwe,2017,1000 ha,100.0
97907,Zimbabwe,2018,1000 ha,100.0
97908,Zimbabwe,2019,1000 ha,100.0
97909,Zimbabwe,2020,1000 ha,100.0


In [49]:
## Export the permanent crops data value to csv
land_use_permanentCrops_df.to_csv('my_data/agricultural_permanent_crops.csv', index = False)

## Food security pre-processing
### Per capita food production variability feature

In [50]:
## Import exchange currency per dollar data

food_security_path = "data/Food security indicators  - FAOSTAT_data_en_2-22-2024.csv"

## Read the csv file and transform to dataframe
food_security_df = pd.read_csv(food_security_path)

## Display the 5 rows of the DataFrame
food_security_df.head(2)

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,FS,Suite of Food Security Indicators,4,Afghanistan,6121,Value,21010,Average dietary energy supply adequacy (percen...,20002002,2000-2002,%,88.0,E,Estimated value,NaN
1,FS,Suite of Food Security Indicators,4,Afghanistan,6121,Value,21010,Average dietary energy supply adequacy (percen...,20012003,2001-2003,%,89.0,E,Estimated value,NaN


In [51]:
# Assuming 'column_name' is the name of the column you're interested in
unique_values = food_security_df['Item'].unique()

# Print the unique values
print(unique_values)

['Average dietary energy supply adequacy (percent) (3-year average)'
 'Average protein supply (g/cap/day) (3-year average)'
 'Cereal import dependency ratio (percent) (3-year average)'
 'Percent of arable land equipped for irrigation (percent) (3-year average)'
 'Value of food imports in total merchandise exports (percent) (3-year average)'
 'Political stability and absence of violence/terrorism (index)'
 'Per capita food production variability (constant 2014-2016 thousand int$ per capita)'
 'Per capita food supply variability (kcal/cap/day)'
 'Prevalence of anemia among women of reproductive age (15-49 years)'
 'Prevalence of low birthweight (percent)']


In [52]:
## Get the Indicator as feature
food_security_political_stability_df = food_security_df[food_security_df['Item'] == 'Political stability and absence of violence/terrorism (index)'].copy()
food_security_political_stability_df

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
99,FS,Suite of Food Security Indicators,4,Afghanistan,6125,Value,21032,Political stability and absence of violence/te...,2000,2000,Index,-2.44,X,Figure from international organizations,NaN
100,FS,Suite of Food Security Indicators,4,Afghanistan,6125,Value,21032,Political stability and absence of violence/te...,2002,2002,Index,-2.04,X,Figure from international organizations,NaN
101,FS,Suite of Food Security Indicators,4,Afghanistan,6125,Value,21032,Political stability and absence of violence/te...,2003,2003,Index,-2.20,X,Figure from international organizations,NaN
102,FS,Suite of Food Security Indicators,4,Afghanistan,6125,Value,21032,Political stability and absence of violence/te...,2004,2004,Index,-2.30,X,Figure from international organizations,NaN
103,FS,Suite of Food Security Indicators,4,Afghanistan,6125,Value,21032,Political stability and absence of violence/te...,2005,2005,Index,-2.07,X,Figure from international organizations,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36424,FS,Suite of Food Security Indicators,716,Zimbabwe,6125,Value,21032,Political stability and absence of violence/te...,2017,2017,Index,-0.71,X,Figure from international organizations,NaN
36425,FS,Suite of Food Security Indicators,716,Zimbabwe,6125,Value,21032,Political stability and absence of violence/te...,2018,2018,Index,-0.72,X,Figure from international organizations,NaN
36426,FS,Suite of Food Security Indicators,716,Zimbabwe,6125,Value,21032,Political stability and absence of violence/te...,2019,2019,Index,-0.96,X,Figure from international organizations,NaN
36427,FS,Suite of Food Security Indicators,716,Zimbabwe,6125,Value,21032,Political stability and absence of violence/te...,2020,2020,Index,-1.07,X,Figure from international organizations,NaN


In [53]:
columns_dropped_food_security = ["Domain Code", "Domain", "Area Code (M49)", "Element Code", "Element", "Item Code", "Item",
                              "Year Code", "Flag", "Flag Description", "Note"]

food_security_political_stability_df = food_security_political_stability_df.drop(columns=columns_dropped_food_security)
food_security_political_stability_df

,Area,Year,Unit,Value
99,Afghanistan,2000,Index,-2.44
100,Afghanistan,2002,Index,-2.04
101,Afghanistan,2003,Index,-2.20
102,Afghanistan,2004,Index,-2.30
103,Afghanistan,2005,Index,-2.07
...,...,...,...,...
36424,Zimbabwe,2017,Index,-0.71
36425,Zimbabwe,2018,Index,-0.72
36426,Zimbabwe,2019,Index,-0.96
36427,Zimbabwe,2020,Index,-1.07


In [54]:
## Export the permanent crops data value to csv
food_security_political_stability_df.to_csv('my_data/political_stability.csv', index = False)

## Foreign direct investment pre-processing
### Total FDI inflows feature

In [55]:
## Import exchange currency per dollar data

foreign_investment_path = "data/Foreign direct investment - FAOSTAT_data_en_2-27-2024.csv"

## Read the csv file and transform to dataframe
foreign_investment_df = pd.read_csv(foreign_investment_path)

## Display the 5 rows of the DataFrame
foreign_investment_df.head(2)

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,FDI,Foreign Direct Investment (FDI),4,Afghanistan,6110,Value US$,23082,Total FDI inflows,2000,2000,million USD,0.17,X,Figure from international organizations,UNCTAD
1,FDI,Foreign Direct Investment (FDI),4,Afghanistan,6110,Value US$,23082,Total FDI inflows,2001,2001,million USD,0.68,X,Figure from international organizations,UNCTAD


In [56]:
# Assuming 'column_name' is the name of the column you're interested in
unique_values = foreign_investment_df['Item'].unique()

# Print the unique values
print(unique_values)

['Total FDI inflows' 'Total FDI outflows'
 'FDI inflows to Agriculture, Forestry and Fishing'
 'FDI inflows to Food, Beverages and Tobacco'
 'FDI outflows to Agriculture, Forestry and Fishing'
 'FDI outflows to Food, Beverages and Tobacco']


In [57]:
## Get the Indicator as feature
foreign_investment_total_inflows_df = foreign_investment_df[foreign_investment_df['Item'] == 'Total FDI inflows'].copy()
foreign_investment_total_inflows_df

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,FDI,Foreign Direct Investment (FDI),4,Afghanistan,6110,Value US$,23082,Total FDI inflows,2000,2000,million USD,0.170000,X,Figure from international organizations,UNCTAD
1,FDI,Foreign Direct Investment (FDI),4,Afghanistan,6110,Value US$,23082,Total FDI inflows,2001,2001,million USD,0.680000,X,Figure from international organizations,UNCTAD
2,FDI,Foreign Direct Investment (FDI),4,Afghanistan,6110,Value US$,23082,Total FDI inflows,2002,2002,million USD,50.000000,X,Figure from international organizations,UNCTAD
3,FDI,Foreign Direct Investment (FDI),4,Afghanistan,6110,Value US$,23082,Total FDI inflows,2003,2003,million USD,57.800000,X,Figure from international organizations,UNCTAD
4,FDI,Foreign Direct Investment (FDI),4,Afghanistan,6110,Value US$,23082,Total FDI inflows,2004,2004,million USD,186.900000,X,Figure from international organizations,UNCTAD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12249,FDI,Foreign Direct Investment (FDI),716,Zimbabwe,6110,Value US$,23082,Total FDI inflows,2018,2018,million USD,745.007943,X,Figure from international organizations,UNCTAD
12250,FDI,Foreign Direct Investment (FDI),716,Zimbabwe,6110,Value US$,23082,Total FDI inflows,2019,2019,million USD,280.000000,X,Figure from international organizations,UNCTAD
12251,FDI,Foreign Direct Investment (FDI),716,Zimbabwe,6110,Value US$,23082,Total FDI inflows,2020,2020,million USD,194.000000,X,Figure from international organizations,UNCTAD
12252,FDI,Foreign Direct Investment (FDI),716,Zimbabwe,6110,Value US$,23082,Total FDI inflows,2021,2021,million USD,250.000000,X,Figure from international organizations,UNCTAD


In [58]:
columns_dropped_foreign_investment = ["Domain Code", "Domain", "Area Code (M49)", "Element Code", "Element", "Item Code", "Item",
                              "Year Code", "Flag", "Flag Description", "Note"]

foreign_investment_total_inflows_df = foreign_investment_total_inflows_df.drop(columns=columns_dropped_foreign_investment)
foreign_investment_total_inflows_df

,Area,Year,Unit,Value
0,Afghanistan,2000,million USD,0.170000
1,Afghanistan,2001,million USD,0.680000
2,Afghanistan,2002,million USD,50.000000
3,Afghanistan,2003,million USD,57.800000
4,Afghanistan,2004,million USD,186.900000
...,...,...,...,...
12249,Zimbabwe,2018,million USD,745.007943
12250,Zimbabwe,2019,million USD,280.000000
12251,Zimbabwe,2020,million USD,194.000000
12252,Zimbabwe,2021,million USD,250.000000


In [59]:
## Export the total inflows data value to csv
foreign_investment_total_inflows_df.to_csv('my_data/fdi_total_inflows.csv', index = False)

### Total FDI outflows feature

In [60]:
## Get the Indicator as feature
foreign_investment_total_outflows_df = foreign_investment_df[foreign_investment_df['Item'] == 'Total FDI outflows'].copy()
foreign_investment_total_outflows_df

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code,Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
22,FDI,Foreign Direct Investment (FDI),4,Afghanistan,6110,Value US$,23085,Total FDI outflows,2003,2003,million USD,1.000000,X,Figure from international organizations,UNCTAD
23,FDI,Foreign Direct Investment (FDI),4,Afghanistan,6110,Value US$,23085,Total FDI outflows,2004,2004,million USD,-0.700000,X,Figure from international organizations,UNCTAD
24,FDI,Foreign Direct Investment (FDI),4,Afghanistan,6110,Value US$,23085,Total FDI outflows,2005,2005,million USD,1.500000,X,Figure from international organizations,UNCTAD
25,FDI,Foreign Direct Investment (FDI),4,Afghanistan,6110,Value US$,23085,Total FDI outflows,2008,2008,million USD,-1.918036,X,Figure from international organizations,UNCTAD
26,FDI,Foreign Direct Investment (FDI),4,Afghanistan,6110,Value US$,23085,Total FDI outflows,2009,2009,million USD,0.334959,X,Figure from international organizations,UNCTAD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12271,FDI,Foreign Direct Investment (FDI),716,Zimbabwe,6110,Value US$,23085,Total FDI outflows,2018,2018,million USD,26.771877,X,Figure from international organizations,UNCTAD
12272,FDI,Foreign Direct Investment (FDI),716,Zimbabwe,6110,Value US$,23085,Total FDI outflows,2019,2019,million USD,32.000000,X,Figure from international organizations,UNCTAD
12273,FDI,Foreign Direct Investment (FDI),716,Zimbabwe,6110,Value US$,23085,Total FDI outflows,2020,2020,million USD,33.000000,X,Figure from international organizations,UNCTAD
12274,FDI,Foreign Direct Investment (FDI),716,Zimbabwe,6110,Value US$,23085,Total FDI outflows,2021,2021,million USD,32.000000,X,Figure from international organizations,UNCTAD


In [61]:
columns_dropped_foreign_investment = ["Domain Code", "Domain", "Area Code (M49)", "Element Code", "Element", "Item Code", "Item",
                              "Year Code", "Flag", "Flag Description", "Note"]

foreign_investment_total_outflows_df = foreign_investment_total_outflows_df.drop(columns=columns_dropped_foreign_investment)
foreign_investment_total_outflows_df

,Area,Year,Unit,Value
22,Afghanistan,2003,million USD,1.000000
23,Afghanistan,2004,million USD,-0.700000
24,Afghanistan,2005,million USD,1.500000
25,Afghanistan,2008,million USD,-1.918036
26,Afghanistan,2009,million USD,0.334959
...,...,...,...,...
12271,Zimbabwe,2018,million USD,26.771877
12272,Zimbabwe,2019,million USD,32.000000
12273,Zimbabwe,2020,million USD,33.000000
12274,Zimbabwe,2021,million USD,32.000000


In [62]:
## Export the total outflows data value to csv
foreign_investment_total_outflows_df.to_csv('my_data/fdi_total_outflows.csv', index = False)